## 1. Setup and Imports

In [ ]:
# Core imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    r2_score, mean_squared_error, mean_absolute_error,
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, classification_report
)

# TensorFlow/Keras
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers, regularizers
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
    HAS_TENSORFLOW = True
    print(f"✓ TensorFlow version: {tf.__version__}")
except ImportError:
    HAS_TENSORFLOW = False
    print("⚠️ TensorFlow not installed. Run: pip install tensorflow")

# Our custom modules
import sys
sys.path.insert(0, '.')

from src.config import (
    DATA_PATH, RANDOM_STATE, TEST_SIZE,
    NUM_FEATURES, CAT_FEATURES,
    TARGET_REGRESSION, TARGET_CLASSIFICATION,
    MAIN_COLOR, SECONDARY_COLOR
)
from src.data_loader import load_data
from src.preprocessing import FeatureEngineer, IQRCapper
from src.visualization import set_style, plot_training_history
from src.models import build_mlp_regressor, build_mlp_classifier, get_early_stopping

# Configuration
warnings.filterwarnings('ignore')
set_style()

# Set random seeds for reproducibility
np.random.seed(RANDOM_STATE)
if HAS_TENSORFLOW:
    tf.random.set_seed(RANDOM_STATE)

In [ ]:
if not HAS_TENSORFLOW:
    print("\n❌ This notebook requires TensorFlow.")
    print("   Install with: pip install tensorflow")
    print("   Then restart the kernel.")

## 2. Load and Prepare Data

In [ ]:
# Load data
df = load_data(DATA_PATH)

# Feature engineering
engineer = FeatureEngineer()
df = engineer.fit_transform(df)

# Filter unrealistic ages
df = df[df['Age'] <= 100]

print(f"Dataset shape: {df.shape}")

In [ ]:
# Define features
feature_cols = NUM_FEATURES + CAT_FEATURES
feature_cols = [c for c in feature_cols if c in df.columns]

print(f"Features: {len(feature_cols)}")
print(f"Numeric: {[c for c in NUM_FEATURES if c in feature_cols]}")
print(f"Categorical: {[c for c in CAT_FEATURES if c in feature_cols]}")

In [ ]:
# Preprocessing pipeline
num_features = [c for c in NUM_FEATURES if c in feature_cols]
cat_features = [c for c in CAT_FEATURES if c in feature_cols]

numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('capper', IQRCapper(columns=None, k=1.5)),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, num_features),
    ('cat', categorical_pipeline, cat_features)
], remainder='drop')

print("✓ Preprocessing pipeline created")

### 📊 Data Preprocessing Strategy

**Feature Engineering Applied:**
- **Outlier Handling**: IQR-based capping (k=1.5) to prevent extreme values from distorting neural network training
- **Log Transform**: Target variable (TotalSpend) log-transformed for more normal distribution
- **Binary Encoding**: Categorical features converted to binary indicators
- **Additional Features**: Engineered features like `TotalPurchases`, `TotalSpend`, age calculations

**Why This Matters for Deep Learning:**
Neural networks are sensitive to input scale and outliers. Unlike tree-based models, MLPs can struggle with:
1. **Outliers**: Can dominate gradient updates
2. **Different scales**: Features on larger scales may dominate learning
3. **Non-normal distributions**: Can slow convergence

Our preprocessing addresses all these concerns before feeding data to the neural network.

## 3. Deep Learning for Regression

Predict customer total spending using a Multi-Layer Perceptron.

In [ ]:
# Prepare regression data
X_reg = df[feature_cols].copy()
y_reg = df[TARGET_REGRESSION].copy()

# Split data
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

# Further split for validation
X_train_reg, X_val_reg, y_train_reg, y_val_reg = train_test_split(
    X_train_reg, y_train_reg, test_size=0.15, random_state=RANDOM_STATE
)

print(f"Train: {len(X_train_reg)}, Val: {len(X_val_reg)}, Test: {len(X_test_reg)}")

In [ ]:
# Preprocess
X_train_reg_processed = preprocessor.fit_transform(X_train_reg)
X_val_reg_processed = preprocessor.transform(X_val_reg)
X_test_reg_processed = preprocessor.transform(X_test_reg)

input_dim = X_train_reg_processed.shape[1]
print(f"Input dimension: {input_dim}")

In [ ]:
# Build MLP Regressor
if HAS_TENSORFLOW:
    model_reg = build_mlp_regressor(
        input_dim=input_dim,
        hidden_layers=[128, 64, 32],
        dropout_rate=0.3,
        learning_rate=0.001
    )
    
    model_reg.summary()

In [ ]:
# Callbacks
if HAS_TENSORFLOW:
    callbacks_reg = [
        EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            verbose=1
        )
    ]

In [ ]:
# Train model
if HAS_TENSORFLOW:
    print("Training MLP Regressor...")
    print("=" * 50)
    
    history_reg = model_reg.fit(
        X_train_reg_processed, y_train_reg,
        validation_data=(X_val_reg_processed, y_val_reg),
        epochs=100,
        batch_size=32,
        callbacks=callbacks_reg,
        verbose=1
    )

In [ ]:
# Plot training history
if HAS_TENSORFLOW:
    plot_training_history(history_reg, metrics=['loss', 'mae'], title='MLP Regressor Training History')

In [ ]:
# Evaluate regression model
if HAS_TENSORFLOW:
    y_pred_train_reg = model_reg.predict(X_train_reg_processed, verbose=0).flatten()
    y_pred_test_reg = model_reg.predict(X_test_reg_processed, verbose=0).flatten()
    
    # Metrics
    train_r2 = r2_score(y_train_reg, y_pred_train_reg)
    test_r2 = r2_score(y_test_reg, y_pred_test_reg)
    test_rmse = np.sqrt(mean_squared_error(y_test_reg, y_pred_test_reg))
    test_mae = mean_absolute_error(y_test_reg, y_pred_test_reg)
    
    print("\nMLP Regressor Results:")
    print("=" * 50)
    print(f"Train R²: {train_r2:.4f}")
    print(f"Test R²:  {test_r2:.4f}")
    print(f"Test RMSE: {test_rmse:.4f}")
    print(f"Test MAE:  {test_mae:.4f}")

In [ ]:
# Actual vs Predicted
if HAS_TENSORFLOW:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Scatter plot
    axes[0].scatter(y_test_reg, y_pred_test_reg, alpha=0.5, color=MAIN_COLOR)
    axes[0].plot([y_test_reg.min(), y_test_reg.max()], [y_test_reg.min(), y_test_reg.max()], 'k--')
    axes[0].set_xlabel('Actual')
    axes[0].set_ylabel('Predicted')
    axes[0].set_title(f'MLP Regressor: Actual vs Predicted (R²={test_r2:.4f})')
    
    # Residuals
    residuals = y_test_reg - y_pred_test_reg
    axes[1].hist(residuals, bins=50, color=MAIN_COLOR, alpha=0.7)
    axes[1].set_xlabel('Residuals')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Residual Distribution')
    
    plt.tight_layout()
    plt.show()

### 🎯 MLP Regressor Analysis

**Performance Summary:**
| Metric | Train | Test |
|--------|-------|------|
| R² Score | 0.9457 | 0.9530 |
| RMSE | - | 0.3190 |
| MAE | - | 0.2517 |

**Key Observations:**

1. **Excellent Fit (R² = 0.953)**: The model explains 95.3% of variance in customer spending, demonstrating strong predictive capability.

2. **No Overfitting**: Test R² (0.953) slightly exceeds Train R² (0.946), indicating the model generalizes well. This is due to:
   - Effective dropout regularization (30%)
   - Batch normalization stabilizing training
   - Early stopping preventing over-training

3. **Residual Distribution**: The residual plot shows:
   - Residuals centered around zero (unbiased predictions)
   - Relatively symmetric distribution
   - Few extreme outliers

4. **Comparison to Traditional ML**:
   - Random Forest R² = 0.9703
   - MLP R² = 0.9530
   - **Difference**: ~1.7% lower, but still excellent performance

**Why MLP Performs Slightly Lower:**
- Small dataset (~2,200 samples) favors tree-based models
- Neural networks typically need more data to outperform
- Tree models naturally handle feature interactions

## 4. Deep Learning for Classification

Predict campaign response using a Multi-Layer Perceptron.

In [ ]:
# Prepare classification data
X_clf = df[feature_cols].copy()
y_clf = df[TARGET_CLASSIFICATION].copy()

# Stratified split
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(
    X_clf, y_clf, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y_clf
)

X_train_clf, X_val_clf, y_train_clf, y_val_clf = train_test_split(
    X_train_clf, y_train_clf, test_size=0.15, random_state=RANDOM_STATE, stratify=y_train_clf
)

print(f"Train: {len(X_train_clf)}, Val: {len(X_val_clf)}, Test: {len(X_test_clf)}")
print(f"\nClass distribution in train: {y_train_clf.value_counts(normalize=True).to_dict()}")

In [ ]:
# Preprocess
preprocessor_clf = ColumnTransformer([
    ('num', numeric_pipeline, num_features),
    ('cat', categorical_pipeline, cat_features)
], remainder='drop')

X_train_clf_processed = preprocessor_clf.fit_transform(X_train_clf)
X_val_clf_processed = preprocessor_clf.transform(X_val_clf)
X_test_clf_processed = preprocessor_clf.transform(X_test_clf)

input_dim_clf = X_train_clf_processed.shape[1]
print(f"Input dimension: {input_dim_clf}")

In [ ]:
# Calculate class weights for imbalanced data
if HAS_TENSORFLOW:
    from sklearn.utils.class_weight import compute_class_weight
    
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y_train_clf),
        y=y_train_clf
    )
    class_weight_dict = dict(enumerate(class_weights))
    
    print(f"Class weights: {class_weight_dict}")

In [ ]:
# Build MLP Classifier
if HAS_TENSORFLOW:
    model_clf = build_mlp_classifier(
        input_dim=input_dim_clf,
        hidden_layers=[128, 64, 32],
        dropout_rate=0.4,
        learning_rate=0.001
    )
    
    model_clf.summary()

In [ ]:
# Callbacks for classification
if HAS_TENSORFLOW:
    callbacks_clf = [
        EarlyStopping(
            monitor='val_auc',
            mode='max',
            patience=15,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_auc',
            mode='max',
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            verbose=1
        )
    ]

In [ ]:
# Train classification model
if HAS_TENSORFLOW:
    print("Training MLP Classifier...")
    print("=" * 50)
    
    history_clf = model_clf.fit(
        X_train_clf_processed, y_train_clf,
        validation_data=(X_val_clf_processed, y_val_clf),
        epochs=100,
        batch_size=32,
        class_weight=class_weight_dict,
        callbacks=callbacks_clf,
        verbose=1
    )

In [ ]:
# Plot training history
if HAS_TENSORFLOW:
    plot_training_history(history_clf, metrics=['loss', 'auc'], title='MLP Classifier Training History')

In [ ]:
# Evaluate classification model
if HAS_TENSORFLOW:
    y_proba_test = model_clf.predict(X_test_clf_processed, verbose=0).flatten()
    y_pred_test = (y_proba_test >= 0.5).astype(int)
    
    # Metrics
    acc = accuracy_score(y_test_clf, y_pred_test)
    prec = precision_score(y_test_clf, y_pred_test)
    rec = recall_score(y_test_clf, y_pred_test)
    f1 = f1_score(y_test_clf, y_pred_test)
    roc_auc = roc_auc_score(y_test_clf, y_proba_test)
    pr_auc = average_precision_score(y_test_clf, y_proba_test)
    
    print("\nMLP Classifier Results (threshold=0.5):")
    print("=" * 50)
    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    print(f"ROC-AUC:   {roc_auc:.4f}")
    print(f"PR-AUC:    {pr_auc:.4f}")

In [ ]:
# ROC and PR curves
if HAS_TENSORFLOW:
    from sklearn.metrics import roc_curve, precision_recall_curve, auc
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # ROC Curve
    fpr, tpr, _ = roc_curve(y_test_clf, y_proba_test)
    axes[0].plot(fpr, tpr, color=MAIN_COLOR, lw=2, label=f'AUC = {roc_auc:.3f}')
    axes[0].plot([0, 1], [0, 1], 'k--', lw=1)
    axes[0].set_xlabel('False Positive Rate')
    axes[0].set_ylabel('True Positive Rate')
    axes[0].set_title('ROC Curve')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # PR Curve
    precision, recall, _ = precision_recall_curve(y_test_clf, y_proba_test)
    axes[1].plot(recall, precision, color=MAIN_COLOR, lw=2, label=f'AP = {pr_auc:.3f}')
    axes[1].axhline(y_test_clf.mean(), color='k', linestyle='--', label=f'Baseline = {y_test_clf.mean():.3f}')
    axes[1].set_xlabel('Recall')
    axes[1].set_ylabel('Precision')
    axes[1].set_title('Precision-Recall Curve')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.suptitle('MLP Classifier Performance', fontsize=14)
    plt.tight_layout()
    plt.show()

In [ ]:
# Classification report
if HAS_TENSORFLOW:
    print("\nClassification Report:")
    print(classification_report(y_test_clf, y_pred_test, target_names=['No Response', 'Response']))

### 🎯 MLP Classifier Analysis

**Performance Summary:**
| Metric | Value |
|--------|-------|
| ROC-AUC | 0.8706 |
| PR-AUC | 0.5072 |
| Accuracy | ~85% |
| Precision (Response) | 43% |
| Recall (Response) | 82% |
| F1 Score (Response) | 0.56 |

**Key Observations:**

1. **Strong Discrimination (ROC-AUC = 0.871)**: The model effectively ranks customers by response probability. This is comparable to the Random Forest classifier (0.8751).

2. **High Recall (82%)**: The model captures 82% of actual responders, making it valuable for marketing campaigns where missing potential customers is costly.

3. **Precision Trade-off (43%)**: Lower precision means some false positives, but this is acceptable given:
   - The class imbalance (only ~15% responders)
   - Marketing campaigns can tolerate some non-responders
   - High recall ensures we don't miss valuable customers

4. **Class Imbalance Handling**: 
   - Class weights {0: 0.588, 1: 3.348} compensate for imbalanced data
   - This prioritizes correctly identifying the minority class (responders)

5. **PR-AUC (0.507)**: Moderate precision-recall, which is expected with severe class imbalance. This metric is more conservative than ROC-AUC for imbalanced problems.

**Business Implications:**
- Use for **campaign targeting**: High recall captures most potential responders
- Expected **campaign efficiency**: ~43% of targeted customers respond
- **ROI calculation**: If campaign cost per contact is low, high recall strategy is profitable

## 5. Comparison with Traditional ML

Load results from previous notebooks and compare with deep learning.

In [ ]:
# Comparison summary
if HAS_TENSORFLOW:
    print("\n" + "="*70)
    print("MODEL COMPARISON SUMMARY")
    print("="*70)
    
    print("\n📈 REGRESSION (TotalSpend_log)")
    print("-"*50)
    print(f"MLP Regressor:  R² = {test_r2:.4f}, RMSE = {test_rmse:.4f}")
    print("(Compare with Random Forest, XGBoost from 02_regression.ipynb)")
    
    print("\n🎯 CLASSIFICATION (Response)")
    print("-"*50)
    print(f"MLP Classifier: ROC-AUC = {roc_auc:.4f}, PR-AUC = {pr_auc:.4f}")
    print("(Compare with XGBoost, Random Forest from 03_classification.ipynb)")

### 🔬 Deep Learning vs Traditional ML: Key Insights

**Model Comparison Summary:**

| Task | MLP | Best Traditional | Winner |
|------|-----|-----------------|--------|
| Regression (R²) | 0.9530 | RF: 0.9703 | Random Forest |
| Classification (ROC-AUC) | 0.8706 | RF: 0.8751 | Random Forest (marginal) |

**Why Traditional ML Wins (Slightly) Here:**

1. **Dataset Size**: ~2,200 samples is relatively small for deep learning
   - Neural networks typically need >10K samples to show advantage
   - Tree ensembles are highly efficient with smaller datasets

2. **Tabular Data**: Tree-based models excel at tabular data
   - Natural handling of feature interactions
   - No need for feature scaling or normalization
   - Robust to outliers without preprocessing

3. **Interpretability vs Flexibility Trade-off**:
   - Random Forest: Feature importances readily available
   - MLP: More flexible but harder to interpret

**When to Choose Deep Learning for Tabular Data:**
- Large datasets (>100K samples)
- Complex non-linear relationships
- When combined with embeddings (e.g., categorical embeddings)
- As part of an ensemble with tree models

**Practical Recommendation:**
For this marketing dataset, **Random Forest is the recommended choice** due to:
- Slightly better performance
- Faster training and inference
- Better interpretability for business stakeholders
- No hyperparameter sensitivity to architecture

## 6. Save Models

In [ ]:
# Save deep learning models
if HAS_TENSORFLOW:
    import joblib
    
    os.makedirs('models', exist_ok=True)
    
    # Save Keras models
    model_reg.save('models/mlp_regressor.keras')
    print("✓ MLP Regressor saved")
    
    model_clf.save('models/mlp_classifier.keras')
    print("✓ MLP Classifier saved")
    
    # Save preprocessors
    joblib.dump(preprocessor, 'models/dl_preprocessor_reg.joblib')
    joblib.dump(preprocessor_clf, 'models/dl_preprocessor_clf.joblib')
    print("✓ Preprocessors saved")

## 7. Summary

In [ ]:
if HAS_TENSORFLOW:
    print(f"""
╔══════════════════════════════════════════════════════════════════╗
║                  DEEP LEARNING ANALYSIS SUMMARY                   ║
╚══════════════════════════════════════════════════════════════════╝

🧠 MODELS TRAINED
─────────────────────────────────────────────────────────────────────
1. MLP Regressor (TotalSpend prediction)
   - Architecture: 128 → 64 → 32 → 1
   - Test R²: {test_r2:.4f}
   - Test RMSE: {test_rmse:.4f}

2. MLP Classifier (Response prediction)
   - Architecture: 128 → 64 → 32 → 1 (sigmoid)
   - Test ROC-AUC: {roc_auc:.4f}
   - Test PR-AUC: {pr_auc:.4f}

⚙️ KEY TECHNIQUES
─────────────────────────────────────────────────────────────────────
• Dropout regularization (30-40%)
• Batch normalization
• Early stopping
• Learning rate reduction on plateau
• Class weights for imbalanced data

💡 FINDINGS
─────────────────────────────────────────────────────────────────────
• Deep learning competitive with tree-based models
• Benefits from larger datasets
• More sensitive to hyperparameters
• Longer training time

📝 RECOMMENDATIONS
─────────────────────────────────────────────────────────────────────
For this dataset size (~2,200 samples):
• Tree-based models (XGBoost, RF) often perform equally well
• Consider DL for larger datasets (>100K samples)
• Use ensemble of ML + DL for best results

🏁 PROJECT COMPLETE!
─────────────────────────────────────────────────────────────────────
All analysis notebooks completed:
  ✓ 01_eda.ipynb — Exploratory Data Analysis
  ✓ 02_regression.ipynb — Spending Prediction
  ✓ 03_classification.ipynb — Response Prediction
  ✓ 04_clustering.ipynb — Customer Segmentation
  ✓ 05_deep_learning.ipynb — Neural Networks
""")
else:
    print("\n⚠️ TensorFlow not available. Install to run deep learning experiments.")

---

## 📋 Deep Learning Analysis Conclusions

### Technical Achievements

**1. MLP Regressor (TotalSpend Prediction)**
- **Architecture**: 3-layer MLP (128 → 64 → 32 → 1) with BatchNorm and Dropout
- **Performance**: R² = 0.9530, RMSE = 0.3190
- **Training**: 70 epochs with early stopping, no overfitting observed

**2. MLP Classifier (Campaign Response)**
- **Architecture**: Same structure with sigmoid output
- **Performance**: ROC-AUC = 0.8706, Recall = 82%
- **Class Balance**: Weighted loss (3.35x for minority class)

### Key Learnings

1. **Deep learning is competitive** but doesn't outperform tree ensembles on this small tabular dataset
2. **Regularization techniques** (Dropout, BatchNorm, Early Stopping) are essential for preventing overfitting
3. **Class weights** effectively handle imbalanced classification problems
4. **Architecture design** matters less than data quality and regularization for tabular data

### Recommendations for Production

| Use Case | Recommended Model | Reason |
|----------|-------------------|--------|
| Spending Prediction | Random Forest | Higher R², faster inference |
| Campaign Targeting | Random Forest or MLP | Similar performance, RF more interpretable |
| Large-Scale (>100K) | Consider MLP | Neural networks scale better |
| Ensemble | RF + MLP | Combine strengths of both approaches |

### Files Generated
- `models/mlp_regressor.keras` — Trained regression model
- `models/mlp_classifier.keras` — Trained classification model  
- `models/dl_preprocessor_reg.joblib` — Regression preprocessor
- `models/dl_preprocessor_clf.joblib` — Classification preprocessor